# Solving linear systems numerically

Consider the generic system of equations:

a11x1 + a12x2 + ... + a1nxn = b1

a21x1 + ...           a2nxn = b2

.

.

.

an1x1 + ...         + annxn = bn

Linalg to the rescue, we remember this is associated to a square matrix $A$ which entries are the $a_{ij}$ coefficients. The matrix form of this system is
$$A\bold x = \bold b$$
where A is the matrix, x is the column matrix of unknowns, b is the column of known values.

A system only has solutions iff $\det A \ne 0$. The reason linear systems can also use numerical solutions, even though a linear system, if it *does* have solutions, **always has an analytically obtainable solution**, is because for fairly large $n$, analytical solving becomes impractical.

---

For now separate note: we may express the unknowns $x_i$ in an upper-triangular $n \times n$ matrix like
$$x_i = \frac{1}{a_{ii}}(b_i-\sum_{k=i+1}^n a_{ik}x_k)$$

and for lower-triangular:

$$x_i = \frac{1}{a_{ii}}(b_i-\sum_{k=1}^{i-1} a_{ik}x_k)$$

---

In numerically approximated solutions, it's best to order the system of equations by triangularizing it (because apparently not doing so causes severe truncation errors if the magnitude of the coefficients differs notably).

When operating $A$'s rows, the equation we subtract from the rest of the equations is called pivot equation.
We also try to **normalize** each row: what this means is taking the largest coefficient (that is, largest in absolute value) of each equation and dividing it to the whole row.

Once that's done, we rearrange the rows so that we're in *Row Echelon Form (REF)* (IT: portarla alla forma di matrice ridotta/a scalini) (*"scaled pivoting"*).

We pick up the work from the second equation: *normalize* and *rearrange* rows.

Third equation, and so on. This brings $A$ to the **Reduced REF** (IT: matrice fortemente ridotta). This "pivoting" operation takes the name of "scaled partial pivoting" (IT: pivoting parziale scalato). 

This is ***extremely computationally expensive***.



<!-- We assume $L$'s diagonal is composed of 1s. *(Doolittle's method)*

What we do is obtaining the first row of the $U$ matrix (equal to $A$'s first row), then the first column for $L$, then the second row for $U$ and so on. The result is a recursive expression for the coefficients of both $L$ and $U$. -->

## LU factorization (or LU decomposition) method

This is one of the algorithms to solve a linsys numerically. This is just weirdo Gaussian elimination but with extra black magic, as we're basically obtaining 2 reduced matrices. We'll work on the square matrix $A$. We'll use two matrices, (**l**ower-triangular) $L$ and  (**u**pper-triangular) $U$ (hence the name of the algorithm) of which the matrix product returns $A$.

$$A = LU$$

The matrix form of the system is then

$$LU\bold x = \bold b$$

This is only apparently more complex, but we're splitting the problem as follows.

The final goal is having our linear system be redefined like this

$$\begin{cases}U\bold x = \bold z \\ L\bold z = \bold b\end{cases}$$

After calculating $L$ and $U$, since we know $\bold b$, we can then obtain $\bold z$. These will be the new known terms of the first matrix equation. The $i$-th z and x values will be 

$$
z_i = (1/l_{ii}) (b_i - \sum_{k = 1}^{i-1} l_{ik}z_k) \quad\text {with } z_1 = b_1 / l_{11}\\
x_i = (1/u_{ii}) (z_i - \sum_{k = i+1}^n u_{ik}x_{k}) \quad\text {with } x_n = z_n / u_{nn}
$$

which is fairly easy to code as it's yet another for-loop. This is basically the final part of the method: **we first need to obtain the LU matrices.**

$L$ has the shape

$$\begin{pmatrix}
l_{11} & 0 & 0 & \dots & 0 \\

l_{21} & l_{22} & 0 & \dots & \\

l_{31} & l_{32} & l_{33} & 0 & \vdots \\

\vdots &  &  & \ddots & \\

l_{n1} & l_{n2} & \dots &  & l_{nn} \\

\end{pmatrix}$$

while $U$ will be the same shape but "flipped upside down"

$$\begin{pmatrix}
u_{11} & u_{12} & \dots & & u_{1n} \\

0 & u_{22} & u_{23} & \dots & \\

0 & 0 & u_{33} & u_{34} & \vdots \\

\vdots &  \vdots &  & \ddots & u_{n-1,n}\\

0 & 0 & \dots & 0 & u_{nn} \\

\end{pmatrix}$$

$A$ has $n^2$ elements, however the triangular matrices, when "overlapped", amount to $n^2 + n$ elements. This means there's an excess of $n$ unknowns. These are *free parameters*. Conventionally, we set the diagonal of $L$ entirely equal to 1; this is called *Doolittle's method*. The symmetrical alternative is setting $U$'s diagonal to 1 instead; this is called *Crout's method*.

Third alternative: $L_{ii} = U_{ii}$. (*Cholesky method*)

---

Let's go by Doolittle's little whims. This simplifies the formulas we wrote above as 1/lii (or uii) = 1.

Since $A = LU$, all we have to do is calculate the $LU$ product: with each result, we rearrange the terms and obtain $u=$ something, then $l=$ something.

Working out **$A$'s first row:**

a11 = u11

a12 = u12

a1i = u1i

**so u1i = a1i (for i = 1..n)**

**First column:**

$$
a11 = u11 \text{  by definition of the method}  \\

a21 = l21 * u11\\

a31 = l31 * u11\\

ai1 = li1 * u11\\
$$

**so li1 = ai1 / u11 (i = 2..n)**

**$A$'s second row:**

we already know a21

$$a22 = l21 * u12 + u22$$
(l22 is =1 so it's omitted)

from this we can somehow infer

$$u_{2i} = a2i - l21 * u1i  (i=2..n)$$

**Second column:**

$$a32 = l31 * u12 + l32 * u22$$

and $$l32 = 1/u22 (a32 - l31 * u12)$$**in general:** $$li2 = 1/u22 (ai2 - li1 * u12) i = 3..n$$

**Third row**

$$a33 = l31*u13 + l32 * u23 + u33$$

generally speaking $u3i = a3i - \sum_{k=1}^{i-1} l3k * uki$ (i = 3..n)

**Third column**
$$
a43 = l41 * u13 + l42 * u23 + l43 * u33 \\

l43 = 1/u33 (a43 - \sum_{k = 1}^3 l4k * uk3)$$

AND IN GENERAL

$$li3 = 1/u33 (ai3 - \sum_{k=1}^{i-1} lik * uk3)$$ 
(i = 4..n)

---


**Finally: we need to first obtain the rows of U and then the columns of L, alternating** (1st row U, 1st column L, 2nd row U, 2nd column L...)

### Rows of $U = u_{ij}$

$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik} u_{kj} \qquad j = i\dots n$$

### Columns of $L = l_{ij}$

$$l_{ij} = \frac{1}{u_{jj}}\left(a_{ij} - \sum_{k=1}^{j-1} l_{ik}u_{kj} \right) \qquad i = j+1\dots n$$

This second formula can be rewritten:

$$l_{ji} = \frac{1}{u_{ii}}\left(a_{ji} - \sum_{k=1}^{i-1} l_{jk}u_{ki} \right) \qquad j = i+1\dots n$$

This allows the indices to walk together in a more compact nested loop. Once we obtained the LU matrices, just loop back to the top part of the notes to get x and z.

---


# Relaxation methods

Relaxation is what I'd like to be doing right now, but here I am.

**Pros:**
- They can solve *nonlinear* systems as well
- Simple application
- **Usually parallelizable**

**Cons:**
- Solutions often aren't analytically accurate, this is numerical approximation at its "best" (read as: worst)
- Only guarantees convergence on a *very* specific class of matrices
- For arbitrarily large systems of equations this is too much to handle for a single, average household PC

Considering a generic linear system, what we do with relaxation is isolating one of the $x$ variables. For instance, a generic first row:

$$x_1 = \frac{1}{a_{11}}(b_1-a12x2-a13x3\dots a1nx_n)$$

and so on for other rows, having them rearranged like

$$x_i = \frac{1}{a_{ii}}(b_i - \sum_{k\ne i}^n a_{ik} x_k)$$

or separate it into two summations where the final term of the first is j-1 and the second starts from j-1.



We have two variants of this formula:

$$x_i^{(p+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{k = 1}^{i-1} a_{ik} x_k^{(p+1)} - \sum_{k = i+1}^{n} a_{ik} x_k^{(p)}\right) \qquad \text{Gauss-Seidel method}$$
$$x_i^{(p+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{k = 1}^{i-1} a_{ik} x_k^{(p)} - \sum_{k = i+1}^{n} a_{ik} x_k^{(p)}\right) \qquad \text{Jacobi method}$$

where Jacobi is parallelizable, GS is not.

(**remember that the sum above skips the $i$-th column!** so for instance the last, nth row will end on the $a_{n,n-1}x_{n-1}$ term)

**Extra note**: the notation $x_i^{(p)}$ can be read as "the p-th iteration of the value x_i" as we can repeat the calculation of x_i by feeding the previous obtained solution back into the formula to obtain improved approximations.

We have an additional method that's a slight , **weighted** variation on GS:

$$x_i^{(p+1)} = x_i^{(p)}+\frac{w}{a_{ii}}\left(b_i - \sum_{k = 1}^{i-1} a_{ik} x_k^{(p+1)} - \sum_{k = i}^{n} a_{ik} x_k^{(p)}\right) \qquad \text{Successive Over-Relaxation}, 1<w<1.5$$

and this accelerates convergence. However, just like GS, this is not parallelizable.

---

At a glance we can already see one of the restrictions of this operation: **this is only applicable to nonzero-diagonal matrices**, more specifically **diagonally dominant matrices.**

The second restriction is extremely specific: **the matrix must NOT contain a nonreducible null submatrix of size $p \times q$ such that $p+q=n$**.

Even if the restrictions are violated, the methods *might* still converge.

as usual we set the iterative method going by giving a precision (epsilon given by $\max|x-previousx|$), which sets how accurate we want **each** x_i solution to be before moving on to the next, and a starting array (row of zeros for instance)



### Code structure

First, give the initial value, then the matrix of the system, the known terms vector.

The outermost for loop is needed to evaluate the x solutions. The innermost for loop(s?) will evaluate pieces of the formula. Outside all of this will be the while loop to check accuracy.